<h1 align='center'>Welcome to my Notebook</h1>

In [6]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold,KFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [49]:
csl_df = pd.read_csv('../../data/csl/CSL_tytl_PI.csv', index_col=0)
X = csl_df.drop(['trans_loss', 'transfus_yes','Sitenum'], axis=1, inplace=False)
y = csl_df['trans_loss'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,
                                                    random_state=7)

<h1 align='center'>Exploratory Data Analysis</h1>

In [50]:
X

,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,Analgesia,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
41-00002,0,8,8,8,99.0,83,0.0,77,150.0,7,...,0,0,0,0,1,0,37,33,1,1
41-00003,0,6,3,2,2.9,77,70.0,1,105.0,8,...,0,0,0,0,0,0,36,22,1,1
41-00004,0,5,3,2,3.3,65,70.0,1,122.0,8,...,0,0,0,0,0,0,34,31,1,1
41-00005,0,11,2,3,4.0,77,80.0,77,138.0,8,...,0,0,0,0,0,0,31,21,1,1
41-00006,0,11,1,3,5.0,98,100.0,77,134.0,8,...,0,0,0,0,0,0,21,22,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,0,8,8,8,77.0,74,70.0,1,124.4,5,...,0,0,0,0,0,0,19,38,0,1
52-07248,0,8,8,8,99.0,74,0.0,2,124.4,5,...,0,0,0,0,1,0,34,24,1,1
52-07249,0,8,8,8,66.0,74,0.0,77,124.4,9,...,0,0,0,0,1,0,28,23,1,1


In [51]:
csl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185413 entries, 41-00002 to 52-07251
Columns: 197 entries, transfus_yes to Sitenum
dtypes: float64(4), int64(193)
memory usage: 280.1+ MB


In [52]:
X.skew()

Accrete           40.650960
AdmBishop         -1.136977
Admcervpos        -0.455299
Admconsistency    -0.407357
Admcontract        0.231978
                    ...    
version9          27.399359
new_age            0.232310
new_BMI            2.086880
new_high_Age      -0.673167
new_high_BMI      -0.549007
Length: 194, dtype: float64

<h1 align='center'>Categorical Features Data Analysis</h1>

In [11]:
%precision %.3f
#np.set_printoptions(precision=3)
print(f'{np.bincount(y)[1]/len(y):.4f}')
print(f'{np.bincount(y_train)[1]/len(y_train):.4f}')

0.0558
0.0558


In [1]:
fig1, ax1 = plt.subplots()
ax1.pie(csl_df['trans_loss'].value_counts(), labels=['No','Yes'], autopct='%1.1f%%')
plt.show()

NameError: name 'plt' is not defined

In [13]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)
#plt.show()

In [14]:
#fig = px.pie(train,values=train['Response'].value_counts(),names=['Class 0','Class 1'],hole=0.6,labels={0:'Response = 0'},color_discrete_sequence=px.colors.sequential.Sunset)
#fig.show(showlegend=True)

In [2]:
sns.countplot(csl_df['trans_loss'])
plt.show()

NameError: name 'sns' is not defined

We can see from the above visualisations that:

* The Data is highly imbalanced

In [3]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_age'],csl_df['trans_loss'])
plt.show()

NameError: name 'plt' is not defined

In [4]:
sns.boxplot(csl_df['new_age'])
plt.show()

NameError: name 'sns' is not defined

In [5]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_BMI'],csl_df['trans_loss'])
plt.show()

NameError: name 'plt' is not defined

In [6]:
sns.boxplot(csl_df['new_BMI'])
plt.show()

NameError: name 'sns' is not defined

<h1 align='center'>Modelling</h1>

In [20]:
#X = all_features.iloc[:len(train),:]
#X_test = all_features.iloc[len(train):,:]

#kf = StratifiedKFold(n_splits=12,shuffle=True,random_state=42)

We define resplit our train and test set, and set up 12 Fold Stratified Cross Validation. It is important to stratify as this ensures that during our training and validation, we split according to target distribution

In [21]:
#for train_index,val_index in kf.split(X,y):
#    X_train,X_val = X.iloc[train_index],X.iloc[val_index],
#    y_train,y_val = y.iloc[train_index],y.iloc[val_index],

In [53]:
# MWB
print(X_train.shape)
print(X.shape)
print(y.shape)

(129789, 194)
(185413, 194)
(185413,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [54]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [55]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(14482, 194)
(14482,)
[7241 7241]


<h1 align='center'>Basic Gradient Boosting</h1>

In [56]:
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
weights = class_weight.compute_sample_weight('balanced', y=y_rus)
gb_rus.fit(X_rus, y_rus, sample_weight=weights)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[34281 18240]
 [  333  2770]]
              precision    recall  f1-score   support

           0       0.99      0.65      0.79     52521
           1       0.13      0.89      0.23      3103

    accuracy                           0.67     55624
   macro avg       0.56      0.77      0.51     55624
weighted avg       0.94      0.67      0.76     55624

ROC AUC prob1: 0.834765447748743
MCC: 0.25818409664321923
F2: 0.414397702112381
PR_AUC: 0.21455806030068342


In [26]:
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
weights = class_weight.compute_sample_weight('balanced', y=y_rus)
gb_rus.fit(X_rus, y_rus, sample_weight=weights)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[35052 17469]
 [  351  2752]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.89      0.24      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC prob1: 0.8355606209858644
MCC: 0.264460260685193
F2: 0.421659056783011
PR_AUC: 0.21392351150913935


<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [72]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 900)
    subsample = trial.suggest_float('subsample', 0.9, 1.0)
#    max_depth = trial.suggest_int('max_depth',3,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.01,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           max_depth=max_depth, learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True)
    for idx, (X_fold, y_fold) in enumerate(cv.split(X,y)):
        model.fit(X_rus, y_rus)
        y_pred = model.predict(X_test)
        probs = model.predict_proba(X_test)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
        roc = roc_auc_score(y_test, prob1)
        mcc = matthews_corrcoef(y_test, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        avg_precision = average_precision_score(y_test, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
#        print(f'MCC: {mcc}')
        
    print(mcc_list)
    return np.mean(mcc_list)

#sampler = TPESampler(seed=42)
sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=15)
#study.optimize(objective,n_trials=5)

[I 2021-12-08 17:19:30,439] A new study created in memory with name: no-name-a5e8843d-bded-490d-be00-8f5535778a33
[I 2021-12-08 17:19:50,223] Trial 0 finished with value: 0.25460762133180487 and parameters: {'n_estimators': 77, 'subsample': 0.9779918792240114, 'learning_rate': 0.09329775397376976, 'min_samples_leaf': 37}. Best is trial 0 with value: 0.25460762133180487.


[0.2576702239341636, 0.25419342147035245, 0.253608104499992, 0.2529587354227114]


[I 2021-12-08 17:23:18,341] Trial 1 finished with value: 0.2583857062252704 and parameters: {'n_estimators': 881, 'subsample': 0.9538495870410434, 'learning_rate': 0.1052128880953882, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.2583857062252704.


[0.2595008430992248, 0.25843732215470655, 0.25757438688234435, 0.2580302727648059]


[I 2021-12-08 17:24:16,697] Trial 2 finished with value: 0.2632711157902071 and parameters: {'n_estimators': 249, 'subsample': 0.949988250082556, 'learning_rate': 0.1390536992629787, 'min_samples_leaf': 41}. Best is trial 2 with value: 0.2632711157902071.


...

[0.26428853760774773, 0.26325519501364586, 0.26260267284251765, 0.2619276068726481]


[I 2021-12-08 17:46:28,377] Trial 13 finished with value: 0.2618493134683866 and parameters: {'n_estimators': 435, 'subsample': 0.9311646387561345, 'learning_rate': 0.04239288963139944, 'min_samples_leaf': 17}. Best is trial 8 with value: 0.26361199405003427.


[0.26204347880139717, 0.26175563517844336, 0.2609115577125206, 0.26268658218118507]


[I 2021-12-08 17:46:32,496] Trial 14 finished with value: 0.23913165603623585 and parameters: {'n_estimators': 12, 'subsample': 0.9433026578608625, 'learning_rate': 0.07941978605761407, 'min_samples_leaf': 33}. Best is trial 8 with value: 0.26361199405003427.


[0.2433646685739289, 0.24338546495193927, 0.23379470450541995, 0.23598178611365525]
CPU times: user 27min 2s, sys: 493 ms, total: 27min 2s
Wall time: 27min 2s


In [73]:
# No Sitenum - 15
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs

Delmode           0.292269
Intratocolytix    0.227085
ROMmeth           0.051721
Inoxy_incrdose    0.045767
GAmethod          0.040865
                    ...   
HxGIdis           0.000000
Hxcsection        0.000000
CS_FVBAC          0.000000
Lac_Other         0.000000
ART               0.000000
Length: 194, dtype: float64

In [59]:
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs

Delmode           0.305558
Intratocolytix    0.236439
Inoxy_incrdose    0.059772
ROMmeth           0.059228
Hxanemia          0.029768
                    ...   
Hyp_clon          0.000000
MthInd_Mec        0.000000
CS_Finduct        0.000000
HIV               0.000000
ART               0.000000
Length: 194, dtype: float64

In [64]:
gb_params = study.best_params #100 runs - No Sitenum
gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[34814 17707]
 [  345  2758]]
              precision    recall  f1-score   support

           0       0.99      0.66      0.79     52521
           1       0.13      0.89      0.23      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.51     55624
weighted avg       0.94      0.68      0.76     55624

ROC AUC prob1: 0.8360652178825845
MCC: 0.262553186075458
F2: 0.4194421632144052
PR_AUC: 0.22013161611918408


In [68]:
gb_params = study.best_params # 25 runs
gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[35036 17485]
 [  372  2731]]
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     52521
           1       0.14      0.88      0.23      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.77      0.52     55624
weighted avg       0.94      0.68      0.77     55624

ROC AUC prob1: 0.8348099183971732
MCC: 0.2610997150624078
F2: 0.4185055780311388
PR_AUC: 0.2163424531568522


In [65]:
gb_params = study.best_params #5 runs
gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[34814 17707]
 [  345  2758]]
              precision    recall  f1-score   support

           0       0.99      0.66      0.79     52521
           1       0.13      0.89      0.23      3103

    accuracy                           0.68     55624
   macro avg       0.56      0.78      0.51     55624
weighted avg       0.94      0.68      0.76     55624

ROC AUC prob1: 0.8360652178825845
MCC: 0.262553186075458
F2: 0.4194421632144052
PR_AUC: 0.22013161611918408


In [42]:
gb_params # 100 runs

{'n_estimators': 200,
 'subsample': 0.941,
 'learning_rate': 0.106,
 'min_samples_leaf': 6,
 'random_state': 42}

In [69]:
gb_params # 25 runs

{'n_estimators': 182,
 'subsample': 0.914,
 'learning_rate': 0.175,
 'min_samples_leaf': 21,
 'random_state': 42}

In [1]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [2]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

<h1 align='center'>Thanks and make sure to learn!</h1>